In [1]:
from sortedcontainers import SortedDict
import logging
import pandas as pd
from collections import deque
        
class Message:
    """
    Represents a message about an order, its execution, or its deletion.
    """
    def __init__(self, ts, msg_type, side, price, qty, order_id, flag, bist_time=None, asset_name=None):
        self.ts = ts
        self.type = msg_type
        self.side = side
        self.price = price
        self.qty = qty
        self.id = order_id
        self.flag = flag
        self.bist_time = bist_time
        self.asset = asset_name

    @classmethod
    def from_tuple(cls, data_tuple):
        """
        Create a Message object from a tuple.
        """
        # Unpack the tuple
        ts, asset_name, msg_type, side, price, qty, order_id, flag = data_tuple
        # Create and return the Message object, mapping 'asset_name' to the correct parameter
        return cls(ts=ts, msg_type=msg_type, side=side, price=price, qty=qty, order_id=order_id, flag=flag, asset_name=asset_name)

    def __str__(self):
        """
        Returns a string representation of the Message object.
        """
        return f"Message(Timestamp: {self.ts}, Type: {self.type}, Side: {self.side}, " \
               f"Price: {self.price}, Quantity: {self.qty}, Order ID: {self.id}, Flag: {self.flag}, " \
               f"BIST Time: {self.bist_time}, Asset: {self.asset})"



class OrderBook:
    """
    The OrderBook class represents a market order book, where orders from traders are stored.
    """

    #helper function to keep the bids in descending order
    @staticmethod
    def _return_neg(x):
        return -x

    def __init__(self,asset):
        """
        Initialize the order book.
        """
        self._reset_levels_()
        self.prev_ts=0
        self.current_ts=0
        self.asset=asset
        #for check
        """
        self.book={
            'ts':[],
            'askpx':[],
            'askqty':[],
            'askpx2':[],
            'askqty2':[],
            'askpx3':[],
            'askqty3':[],   
            'bidpx':[],
            'bidqty':[],
            'bidpx2':[],
            'bidqty2':[],
            'bidpx3':[],
            'bidqty3':[],
        }
        """

    def get_asset(self):
        return self.asset
    
    def get_ts(self):
        return self.current_ts
    
    def get_Q_size(self,side,price):
        id_by_price = self.bid_ids_by_price if side == 'B' else self.ask_ids_by_price
        q=id_by_price.get(price,[])
        return len(q)

    def _reset_levels_(self):
        self.bid_ids_by_price = SortedDict(OrderBook._return_neg)#<float, queue<int>>
        self.ask_ids_by_price = SortedDict()
        self.orders_by_id = {}  # A dictionary to access all orders by id
        self.bid_qty_by_price = SortedDict(OrderBook._return_neg)
        self.ask_qty_by_price = SortedDict()

    def on_new_message(self, message : Message) -> tuple:
        """
        Method to handle different types of messages and modify the order book accordingly.
        returns a tuple consisting of updated order's queue location, and
        the sum of the quantities at this price level up to the updated order

        returns : (q_loc, order_type, side)
        """

        if message.asset!=self.asset:
            raise ValueError('The book and the message are not for the same asset.')
        self.current_ts=message.ts
        q_loc=None#this is for the case where an oorder is deleted in a prce level and it is in front of our order.It is in format (sum_qty,q_loc)
        if message.type == 'A':
            self.add_order(message)
        elif message.type == 'D':
            q_loc = self.delete_order(message)
        elif message.type == 'E':
            q_loc=self.execute_order(message)
        else:#it is a message about an event in bist
            #self._reset_()
            if message.flag in ['P_GUNSONU','P_ACS_EMR_TP_PY_EIY']:
                self._reset_levels_()
        """
        array(['P_GUNSONU', 'P_ACS_EMR_TP_PY_EIY', 'P_ESLESTIRME', 'NONE',
            'P_MARJ_YAYIN', 'P_ARA', 'P_SUREKLI_ISLEM', 'P_MARJ_YAYIN_KAPANIS',
            'P_KAPANIS_FIY_ISLEM', 'P_GUNSONU_ISLEMLERI', 'P_KAPANIS_EMIR_TPL',
            'P_DK_TEKFIY_EMIR_TPL'], dtype=object)
        """
        self.prev_ts=message.ts
        """
        ##############################
        self.book['ts'].append(message.ts)
        self.book['askpx'].append(self.get_best_ask()[0])
        self.book['askqty'].append(self.get_best_ask()[1])
        self.book['askpx2'].append(self.get_best_ask(1)[0])
        self.book['askqty2'].append(self.get_best_ask(1)[1])
        self.book['askpx3'].append(self.get_best_ask(2)[0])
        self.book['askqty3'].append(self.get_best_ask(2)[1])

        self.book['bidpx'].append(self.get_best_bid()[0])
        self.book['bidqty'].append(self.get_best_bid()[1])
        self.book['bidpx2'].append(self.get_best_bid(1)[0])
        self.book['bidqty2'].append(self.get_best_bid(1)[1])
        self.book['bidpx3'].append(self.get_best_bid(2)[0])
        self.book['bidqty3'].append(self.get_best_bid(2)[1])

        ##############################
        """

        return q_loc

    
    def add_order(self, message):
        """
        Adds an order to the book based on its type.
        returns a tuple consisting of the price level, total number of orders in the price level, total quantity of orders up to the deleted order,
        updated order's queue location, the flag representing the type of the message(D,E,R) A in this case.
        returns : (price, sum_qty, q_loc, 'D') 
        """
        id_by_price = self.bid_ids_by_price if message.side == 'B' else self.ask_ids_by_price
        order_dict = self.orders_by_id
        qty_by_price= self.bid_qty_by_price if message.side == 'B' else self.ask_qty_by_price
        
        if message.price not in list(id_by_price.keys()):
            id_by_price[message.price] = deque()
            qty_by_price[message.price] = 0

        level_q=id_by_price[message.price]
        level_q.append(message.id)
        order_dict[message.id] = message
        qty_by_price[message.price] += message.qty
        
    def delete_order(self, message):
        """
        Deletes an order from the book.
        returns a tuple consisting of the price level, total number of orders in the price level, total quantity of orders up to the deleted order,
        updated order's queue location, the flag representing the type of the message(D,E,R) D in this case.
        
        returns : (price, sum_qty, q_loc, 'D') 
        """
        id_by_price = self.bid_ids_by_price if message.side == 'B' else self.ask_ids_by_price
        order_dict = self.orders_by_id
        qty_by_price = self.bid_qty_by_price if message.side == 'B' else self.ask_qty_by_price
        
        order = order_dict.get(message.id)
        if order is None:
            logging.warning(f"D: Order {message.id} does not exist.")
            return
        #remove the order from the level queue
        level_q=id_by_price[order.price]
        index=0
        q_loc=0
        sum_qty_tmp=0
        sum_qty=0#!I will use it later!
        tempQ=deque()
        while len(level_q)>0:
            next_id=level_q.popleft()
            if next_id==message.id:
                q_loc=index
                sum_qty=sum_qty_tmp
            else:
                sum_qty_tmp+=order_dict[next_id].qty
                tempQ.append(next_id)
            index+=1
        id_by_price[order.price]=tempQ
        
        #delete the order from the order by id in all cases
        qty_by_price[order.price] -= order.qty
        #if this was the last order at this price
        if qty_by_price[order.price]==0:
            del qty_by_price[order.price]
            del id_by_price[order.price]
            
        del order_dict[message.id]
        return q_loc

    def execute_order(self, message):
        id_by_price = self.bid_ids_by_price if message.side == 'B' else self.ask_ids_by_price
        order_dict = self.orders_by_id
        qty_by_price = self.bid_qty_by_price if message.side == 'B' else self.ask_qty_by_price
        
        order=order_dict.get(message.id)
        if order is None:
            #this is possible if we run a taker strategy and the order is already executed
            logging.warning(f"E: Order {message.id} does not exist.")
            return
        qty_by_price[order.price] -= message.qty
        level_q=id_by_price[order.price]
        order.qty-=message.qty
        q_loc=None
        if order.qty==0:
            level_q.remove(message.id)
            del order_dict[message.id]
            q_loc=0
        #if this was the last order at this price
        if qty_by_price[order.price]==0:
            del qty_by_price[order.price]
            del id_by_price[order.price]
        return q_loc

    def _get_qty_by_price(self, side, price):
        """
        Returns the total quantity at a given price on a given side. for checling if the id byprice works correctly
        """
        id_by_price = self.bid_ids_by_price if side == 'B' else self.ask_ids_by_price  
        q=id_by_price.get(price,[])
        qty=0
        for id_ in q:
            qty+=self.orders_by_id[id_].qty
        return qty

    def get_best_bid(self, level=0):
        """
        Returns the best bid price and quantity at a given level.
        """
        if len(self.bid_qty_by_price) > level:
            price = list(self.bid_qty_by_price.keys())[level]
            qty = self.bid_qty_by_price[price]
            return price, qty
        else:
            return None, None
    
    def get_best_ask(self, level=0):
        """
        Returns the best ask price and quantity at a given level.
        """
        if len(self.ask_qty_by_price) > level:
            price = list(self.ask_qty_by_price.keys())[level]
            qty = self.ask_qty_by_price[price]
            return price, qty
        else:
            return None, None


In [40]:
import pandas as  pd
from abc import ABC, abstractmethod
from collections import deque
from sortedcontainers import SortedDict

class OrderRequest:
    def __init__(self, asset, ts):
        self.asset = asset
        self.ts = ts

class AddRequest(OrderRequest):
    def __init__(self, asset, ts, side, price, quantity):
        super().__init__(asset, ts)
        self.side = side
        self.price = price
        self.quantity = quantity

class DeleteRequest(OrderRequest):
    def __init__(self, asset, ts ,order_id, side):
        super().__init__(asset, ts)
        self.order_id = order_id
        self.side = side

class ExecuteRequest(OrderRequest):
    def __init__(self, asset, ts, side, quantity):
        super().__init__(asset, ts)
        self.side = side
        self.quantity = quantity
    
class Order:
    """
    Represents an order with separate timestamps for when the order was sent and when it was received.
    The orders that we receive from the market are represented by the Message class.
    """
    def __init__(self, sent_ts, received_ts, msg_type, side, price, qty, order_id, que_loc, asset_name=None):
        self.sent_ts = sent_ts
        self.received_ts = received_ts
        self.type = msg_type
        self.side = side
        self.price = price
        self.qty = qty
        self.id = order_id
        self.que_loc = que_loc
        self.asset = asset_name

    def __str__(self):
        """
        Returns a string representation of the Order object
        """
        return f"Order(Sent Timestamp: {self.sent_ts}, Received Timestamp: {self.received_ts}, Type: {self.type}, " \
               f"Side: {self.side}, Price: {self.price}, Quantity: {self.qty}, Order ID: {self.id}, " \
               f"Queue Location: {self.que_loc}, Asset: {self.asset})"



class BaseStrategy(ABC):

    def __init__(self, data : pd.DataFrame, assets : list[str], cash : float, inventory : dict[str, float]):
        """
        Initialize the strategy with a list of assets, an initial cash balance, and an initial inventory.
        """
        self.assets = assets
        self.cash = cash
        self.inventory = inventory
        self.lobs : dict[str, OrderBook] = {}
        self.data=data
        self.current_ts=0
        self.requestsQ : dict[str,deque[OrderRequest]] = {}#asset:queue[request]
        self.orders_by_id : dict[str,dict[str, dict[int,Order]]]={}#asset:side:order_id:order
        self.orders_by_price : dict[str,dict[str, SortedDict[float, deque[str]]]]={}#asset:side:price:queue[order_id]
        #keeping track of the orders via this id
        self.id_counter=0
        for asset in assets:
            self.reset_asset(asset)
    
    def reset_asset(self, asset : str):
        self.lobs[asset] = OrderBook(asset)
        self.requestsQ[asset]=deque()
        self.orders_by_id[asset]={}
        self.orders_by_id[asset]['B']={}
        self.orders_by_id[asset]['S']={}
        self.orders_by_price[asset]={}
        self.orders_by_price[asset]['B']=SortedDict(OrderBook._return_neg)
        self.orders_by_price[asset]['S']=SortedDict()

    def get_current_ts(self):
        return self.current_ts
    
    def get_cash(self):
        return self.cash
    
    def get_inventory(self, asset : str=None):
        if asset is None:
            return self.inventory
        else:
            return self.inventory[asset]
    
    def get_orders_at_px(self, asset : str, side : str, price : float):
        Q= self.orders_by_price[asset][side].get(price,[])
        list_of_orders=deque()
        for order_id in Q:
            list_of_orders.append(self.orders_by_id[asset][side][order_id])
        return list_of_orders

    
    def get_all_orders(self, asset : str, side : str):
        return self.orders_by_id[asset][side].values()
    
    def get_order(self, order_id : int, asset : str, side : str):
        return self.orders_by_id[asset][side].get(order_id)
        
    
    def __handle_requests__(self, asset):
        while len(self.requestsQ[asset]) != 0 and self.requestsQ[asset][0].ts<=self.current_ts:
            request = self.requestsQ[asset].popleft()
            if isinstance(request, AddRequest):
                self.__handle_add_request__(request)
            elif isinstance(request, DeleteRequest):
                self.__handle_delete_request__(request)
            elif isinstance(request, ExecuteRequest):
                self.__handle_market_order_request__(request)
            else:
                raise ValueError('Invalid request type.')

    def __handle_add_request__(self, request : AddRequest):
        que_loc=self.get_book(request.asset).get_Q_size(request.side,request.price)
        order=Order(request.ts, self.get_current_ts(),'A', request.side, request.price, request.quantity, self.id_counter, que_loc, request.asset)
        self.id_counter+=1
        self.orders_by_id[request.asset][request.side][order.id]=order
        self.orders_by_price[request.asset][request.side].setdefault(request.price,deque()).append(order.id)
        self.on_transaction(order, 'A')
    
    def __handle_delete_request__(self, request : DeleteRequest):
        try:
            order=self.orders_by_id[request.asset][request.side][request.order_id]
            del self.orders_by_id[request.asset][request.side][request.order_id]
            self.orders_by_price[request.asset][request.side][order.price].remove(request.order_id)
            self.on_transaction(order, 'D')
        except KeyError:
            print('Order not found. Probably already executed.')

    def __handle_market_order_request__(self, request : ExecuteRequest):       
        vwap=0
        qty=0
        if request.side=='B':
            while request.quantity-qty>0:
                available_qty = self.lobs[request.asset].get_best_ask()[1]
                if available_qty==None:
                    print('No available order in the market.')
                    return
                px, lots=self.lobs[request.asset].get_best_ask()[0], min(available_qty, request.quantity)
                if px==None:
                    print('No more orders to execute.')
                    return
                self.cash-=px*lots
                vwap+=px*lots
                qty+=lots
                self.inventory[request.asset]+=lots
        else:
            while request.quantity-qty>0:
                available_qty = self.lobs[request.asset].get_best_bid()[1]
                if available_qty==None:
                    print('No available order in the market.')
                    return
                px, lots=self.lobs[request.asset].get_best_bid()[0], min(available_qty, request.quantity)
                self.cash+=px*lots
                vwap+=px*lots
                qty+=lots
                self.inventory[request.asset]-=lots
        vwap/=qty
        #creating a market order 'M' with the vwap and the quantity
        order=Order(request.ts, self.get_current_ts(),'M', request.side, vwap, qty, self.id_counter, 0, request.asset)
        self.id_counter+=1
        self.on_transaction(order, 'M', qty)
    
    def __handle_messages__(self, Q_loc, message : Message ):
        """
        this method updates the queue locations of the orders and deletes the orders that are executed after calling on_transaction.
        """
        if message.type=='D':
            self.__handle_delete_message__(Q_loc,message)
        elif message.type=='A':
            pass
            #self.__handle_add__(asset, px, sum_qty, q_loc, side)
        elif message.type=='E':
            self.__handle_exec_message__(Q_loc,message)
        elif message.type=='O':#market event
            if message.flag in ['P_GUNSONU','P_ACS_EMR_TP_PY_EIY']:
                self._reset_levels_()
            
        
    def __handle_delete_message__(self,Q_loc,message : Message):
        Q=self.get_orders_at_px(message.asset,message.side,message.price)
        if Q is None:
            return
        tempQ=deque()
        while len(Q)>0:
            next_id=Q.popleft()
            our_order=self.get_order(next_id,message.asset,message.side)
            our_q_loc=our_order.que_loc
            if our_q_loc > Q_loc and our_q_loc>0:
                our_order.que_loc-=1
            tempQ.append(next_id)
        self.orders_by_price[message.asset][message.side][message.price]=tempQ
    """    
    def __handle_add__(self, asset, px, sum_qty, q_loc,side):
        #if an add order of opposite side come to a price level on which we have orders with q_loc=0,
        #we should execute them, otherwise, no need to update
        opp_side='B' if side=='S' else 'S'
        Q=self.orders_by_price[asset][opp_side].get(px)
        if Q is not None and len(Q)>0:
            order=self.orders_by_id[asset][opp_side][Q[0]]                              
            self.__handle_exec_message__(asset, px, sum_qty, q_loc,'B')
    """
    def __handle_exec_message__(self,Q_loc : int, message : Message):
        #if there is an executed order on that price level, we should update the queue locations of the orders
        Q=self.get_orders_at_px(message.asset,message.side,message.price)
        if Q is None or len(Q)==0:
            return
        if Q_loc==0:#it means that the order in the book is fully executed        
            our_order_in_front=Q[0]
            if our_order_in_front.que_loc!=0:#it mens thaty our orders in the Q are behind the order that is executed
                for order in Q:
                    order.que_loc-=1
            else:#it means that our orders in the Q are in front of the order that is executed
                qty_to_be_executed=min(our_order_in_front.qty,message.qty)
                while our_order_in_front.que_loc==0 and qty_to_be_executed>0:
                    #will call self.ontransaction
                    our_order_in_front.qty-=qty_to_be_executed
                    if message.side=='B':
                        self.cash-=message.price*qty_to_be_executed
                        self.inventory[message.asset]+=qty_to_be_executed
                    else:
                        self.cash+=message.price*qty_to_be_executed
                        self.inventory[message.asset]-=qty_to_be_executed
                    self.on_transaction(our_order_in_front,'E',qty_to_be_executed)
                    if our_order_in_front.qty==0:
                        Q.popleft()
                        del self.orders_by_id[message.asset][message.side][our_order_in_front.id]
                    qty_to_be_executed-=our_order_in_front.qty
                    if len(Q)>0:
                        our_order_in_front=Q[0]
                    else:
                        break
        else:#it means that the order in the book is partially executed, I will adjust bthe sum qty
            pass


    def get_book(self, asset : str):
        """
        Get the order book for an asset.
        """
        return self.lobs[asset]
    
    def get_requests(self, asset : str=None):
        """
        Get the requests for an asset.
        """
        if asset is None:
            return self.requestsQ
        else:
            return self.requestsQ[asset]
    
    #These methods are used to add reequests to the queue, they are not directly directed to the lob
    def add_order(self, asset : str, side : str, price : float, quantity : int):
        """
        send an order to the order book.
        """
        if price==None or quantity==None:
            raise ValueError('Price and quantity must not be null.')
        if price<=0 or quantity<=0:
            raise ValueError('Price and quantity must be positive.')
        
        if asset not in self.assets:
            raise ValueError('Asset not found.')
        ##############################!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        self.requestsQ[asset].append(AddRequest(asset, self.current_ts, side, price, quantity))
    
    def delete_order(self, asset : str, order_id : int, side : str):
        """
        Delete an order from the order book.
        """
        if asset not in self.assets:
            raise ValueError('Asset not found.')
        
        if order_id not in self.orders_by_id[asset]['B'] and order_id not in self.orders_by_id[asset]['S']:
            raise ValueError('Order not found.')

        self.requestsQ[asset].append(DeleteRequest(asset, self.current_ts, order_id, side))
    
    def market_order(self, asset : str, side : str, quantity : int):
        """
        Execute a market order.
        """
        if asset not in self.assets:
            raise ValueError('Asset not found.')
        
        if quantity==None:
            raise ValueError('Quantity must not be null.')

        if quantity<=0:
            raise ValueError('Quantity must be positive.')

        self.requestsQ[asset].append(ExecuteRequest(asset, self.current_ts, side, quantity))


    def run(self):
        """
        Run the strategy.
        """
        for row in self.data.itertuples():
            #this tuple will be used for the update the orders and their Q locations
            msg=Message.from_tuple(row)
            book=self.lobs[msg.asset]
            self.__handle_requests__(msg.asset)
            new_Q_loc=book.on_new_message(msg)
            self.current_ts=book.get_ts()
            self.__handle_messages__(new_Q_loc,msg)
            self.on_update(msg)


    @abstractmethod 
    def on_update(self, message : Message):
        """
        this method is called per message for each asset. It is an asbstract method and should be implemented by the user.
        """
        pass
    
    @abstractmethod 
    def on_transaction(self,order : Order, event_type : str, exec_qty : int=None):
        """
        this method is called per trade for each asset. It is an abstract method and should be implemented by the user.
        Params:
        order: the order that is added, deleted or executed
        event_type: 'A', 'D' 'E' 'M' for add, delete, execute, market order(E-->execution of a limit order, M-->execution of a market order)
        """
        pass


from qrd.data.utils import *
file_path = 'messages.txt'
with open(file_path, 'r') as file:
    messages = pd.Series(file.readlines())

# Apply the extract_messages function to create a DataFrame
messages_df = extract_messages(messages)
messages_df['asset'] = "AKBNK"  # Add the 'asset' column
# Create a new column order with 'asset' as the first column
new_column_order = ['asset'] + [col for col in messages_df.columns if col != 'asset']
# Reindex the DataFrame with the new column order
messages_df = messages_df[new_column_order]


class MyStrategy(BaseStrategy):
#    def __init__(self, data : pd.DataFrame, assets : list[str], cash : float, inventory : dict[str, float]):

    def __init__(self, data, assets):
        super().__init__(data, assets,cash=100,inventory={'AKBNK':0})
         

    
    def on_update(self, message : Message):
        """        
        current_ts=self.get_current_ts()    
        start_time = pd.Timestamp(current_ts.date()).replace(hour=10, minute=0, second=0)
        end_time = pd.Timestamp(current_ts.date()).replace(hour=18, minute=0, second=0)

        if current_ts<start_time or current_ts>end_time:
            return
        """
        best_bid_px=self.get_book('AKBNK').get_best_bid()[0]
        best_ask_px=self.get_book('AKBNK').get_best_ask()[0]
        
        if best_bid_px!=None:
            if len(self.get_orders_at_px('AKBNK','B',best_bid_px))==0:
                self.add_order('AKBNK','B',best_bid_px,1)
            #deleting the orders that are not in best bid
            for order in self.get_all_orders('AKBNK','B'):
                if order.price!=best_bid_px:
                    self.delete_order('AKBNK',order.id,'B')
        elif best_ask_px!=None:
            if len(self.get_orders_at_px('AKBNK','S',best_ask_px))==0:
                self.add_order('AKBNK','S',best_ask_px,1)
            #deleting the orders that are not in best ask
            for order in self.get_all_orders('AKBNK','S'):
                if order.price!=best_ask_px:
                    self.delete_order('AKBNK',order.id,'S')

        print(f"------{self.get_current_ts()}------")
        print(message)
        print(self.get_book('AKBNK').get_best_bid())
        print(self.get_book('AKBNK').get_best_ask())
        print(self.get_inventory('AKBNK'))
        print(self.get_cash())
        
        for order in self.get_all_orders('AKBNK','B'):
            print(order)
        for order in self.get_all_orders('AKBNK','S'):
            print(order)
                        
    def on_transaction(self, order: Order, event_type: str, exec_qty : int=None):
        #print(order, "event: ", event_type)
        pass


my_strategy=MyStrategy(messages_df,['AKBNK'])
my_strategy.run()
        
            

------0------
Message(Timestamp: 0, Type: A, Side: S, Price: 12, Quantity: 5, Order ID: 1, Flag: NONE, BIST Time: None, Asset: AKBNK)
(None, None)
(12, 5)
0
100
------1------
Message(Timestamp: 1, Type: A, Side: B, Price: 11, Quantity: 1, Order ID: 2, Flag: NONE, BIST Time: None, Asset: AKBNK)
(11, 1)
(12, 5)
0
100
Order(Sent Timestamp: 0, Received Timestamp: 0, Type: A, Side: S, Price: 12, Quantity: 1, Order ID: 0, Queue Location: 1, Asset: AKBNK)
------2------
Message(Timestamp: 2, Type: E, Side: B, Price: 11, Quantity: 1, Order ID: 2, Flag: NONE, BIST Time: None, Asset: AKBNK)
(None, None)
(12, 5)
0
100
Order(Sent Timestamp: 1, Received Timestamp: 1, Type: A, Side: B, Price: 11, Quantity: 1, Order ID: 1, Queue Location: 0, Asset: AKBNK)
Order(Sent Timestamp: 0, Received Timestamp: 0, Type: A, Side: S, Price: 12, Quantity: 1, Order ID: 0, Queue Location: 1, Asset: AKBNK)
------3------
Message(Timestamp: 3, Type: A, Side: B, Price: 11, Quantity: 1, Order ID: 3, Flag: NONE, BIST Time: 

In [41]:
my_strategy.get_cash()

89

In [42]:
my_strategy.get_inventory()

{'AKBNK': 1}